In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

---
## ⚙️ Section 1: Setup

### 1.1: Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

### 1.2: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [30]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool
from google.adk.agents import ParallelAgent, SequentialAgent
from google.adk.apps.llm_event_summarizer import LlmEventSummarizer

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Helper functions

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [5]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


### 1.5: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Section 2: Replicating 'Fashion advisor' app inside the notebook and exploring context engineering

### 2.1 Implementing Our Fashion App using same agents and gemini model

Let's replicate the code present inside github so that we can explore context engineering


In [21]:
MODEL_NAME = "gemini-2.0-flash"

def get_retail_data() -> Any:
    """
    Gather store retail data.

    Returns:
        Any: store data information
    """
    try:
        data = {
        "StoreSalesData": {
            "Store": {
            "Products": [
                {
                "ProductID": "P001",
                "Name": "Men's Denim Jacket",
                "Category": "Men",
                "Price": 79.99,
                "UnitsSold": 324,
                "Month": "2025-06",
                "ProfitCategory": "Highly Profitable"
                },
                {
                "ProductID": "P002",
                "Name": "Women's Floral Dress",
                "Category": "Women",
                "Price": 49.99,
                "UnitsSold": 512,
                "Month": "2025-06",
                "ProfitCategory": "Highly Profitable"
                },
                {
                "ProductID": "P003",
                "Name": "Unisex White Sneakers",
                "Category": "Footwear",
                "Price": 69.99,
                "UnitsSold": 610,
                "Month": "2025-06",
                "ProfitCategory": "Highly Profitable"
                },
                {
                "ProductID": "P004",
                "Name": "Men's Casual T-Shirt",
                "Category": "Men",
                "Price": 19.99,
                "UnitsSold": 450,
                "Month": "2025-06",
                "ProfitCategory": "Moderately Profitable"
                },
                {
                "ProductID": "P005",
                "Name": "Women's Handbag",
                "Category": "Accessories",
                "Price": 89.99,
                "UnitsSold": 278,
                "Month": "2025-06",
                "ProfitCategory": "Highly Profitable"
                },
                {
                "ProductID": "P006",
                "Name": "Kids' Hoodie",
                "Category": "Kids",
                "Price": 29.99,
                "UnitsSold": 198,
                "Month": "2025-06",
                "ProfitCategory": "Less Profitable"
                },
                {
                "ProductID": "P007",
                "Name": "Women's Running Shoes",
                "Category": "Footwear",
                "Price": 59.99,
                "UnitsSold": 431,
                "Month": "2025-06",
                "ProfitCategory": "Moderately Profitable"
                },
                {
                "ProductID": "P008",
                "Name": "Men's Leather Belt",
                "Category": "Accessories",
                "Price": 24.99,
                "UnitsSold": 150,
                "Month": "2025-06",
                "ProfitCategory": "Less Profitable"
                },
                {
                "ProductID": "P009",
                "Name": "Women's Winter Coat",
                "Category": "Women",
                "Price": 129.99,
                "UnitsSold": 95,
                "Month": "2025-06",
                "ProfitCategory": "Less Profitable"
                },
                {
                "ProductID": "P010",
                "Name": "Unisex Sports Cap",
                "Category": "Accessories",
                "Price": 14.99,
                "UnitsSold": 310,
                "Month": "2025-06",
                "ProfitCategory": "Moderately Profitable"
                }
            ]
            }
        }
        }

        return data

    except Exception as e:
        return {
            "result": {"error": f"Failed to gather retail information: {str(e)}"},
            "stats": {"success": False},
            "additional_info": {"error_type": str(type(e).__name__)},
        }

retail_data_agent = LlmAgent(
    name="RetailDataAgent",
    model=MODEL_NAME,
    instruction="""You are a store retail agent.
    
    Use tool 'get_retail_data' to get the json data about the previous year store retail data.
    You need to analyse the data and provide the summary of the data in final response.
    Make sure that the summary must have two key points:
    
    1. product name
    2. ProfitCategory
    
    IMPORTANT: You MUST call the get_retail_data tool. Do not make up information.
    """,
    description="Gathers and analyzes store retail data",
    tools=[get_retail_data],
    # tools=[retail_data_tool],
    output_key="retail_data",
)

get_latest_trending_fashion_news = Agent(
    name="get_latest_trending_fashion_news",
    model=MODEL_NAME,
    description="News analyst agent to get trending fashion news",
    instruction="""
    You are a helpful assistant that can fetches the news about what .

    Use 'google_search' tool to fetch the current trends.
    You must have answers for below 5 questions:

    1. Which product categories and styles are currently selling the most?
    2. What colors, fabrics, or designs are most popular among customers right now?
    3. What fashion items are trending on social media and influencing customer demand?
    4. Which customer segments (age, gender, region) are driving current trends?
    5. Which products are seeing a rapid increase in sales or searches compared to previous weeks/months?

    IMPORTANT: You MUST call the google_search tool to fetch answers of above 5 questions. Do not make up information.
    """,
    tools=[google_search],
)

trending_fashion_agent = LlmAgent(
    name="TrendingFashionAgent",
    model=MODEL_NAME,
    instruction="""You are a trending data analysis agent.
    
    Use tool 'get_latest_trending_fashion_news' to get all the latest news about fashion.
    You need to analyse the data to create a summary of what are the fashion trends going on and what are the driving factors of those.
    If you face some data missing issue, just create a summary with the data available.
    DO NOT say that the data is not available
    
    IMPORTANT: You MUST call the get_latest_trending_fashion_news tool. Do not make up information.
    """,
    description="Gathers and analyzes memory information",
    tools=[AgentTool(get_latest_trending_fashion_news)],
    output_key="trending_info",
)

# System Report Synthesizer Agent
fashion_report_synthesizer = LlmAgent(
    name="FashionReportSynthesizer",
    model=MODEL_NAME,
    instruction="""You are a Fashion Report Synthesizer.

    You are an helpful recommendation assistant who is helping the owner of a fashion store to know what are in fashion that are currently trending and which are not which will help the owner to increase the sales of the shop.
    
    Your task is to create a detailed repoty by combining information from:
    - Retail Data: {retail_data}
    - Tending news on Fashion: {trending_info}

    You need to analyse all the information about latest_news, trending_fashion and retail_data.
    
    Create a well-formatted report with:
    1. Recommend some products and category of products which are in trend currently that will help owner to increase the sales. Also compare the trending products with the products present in retail data. If found similarity, mention it in the report about that product.
    2. Give a detailed point of view about the products explaining why are they recommended. And also is there anything in the local news that can affect the sales of the products. To get the local news use sub agent 'remote_news_agent'.
    3. At the end give some suggestions about how the sales can be increased. For example, displaying the recommended products, giving some discounts on those products.
    
    Use markdown formatting to make the report readable and professional.
    Highlight any concerning values and provide practical recommendations.
    """,
    description="Synthesizes all information into a comprehensive report",
)

# --- 1. Create Parallel Agent to gather information concurrently ---
system_info_gatherer = ParallelAgent(
    name="system_info_gatherer",
    sub_agents=[retail_data_agent, trending_fashion_agent],
    # sub_agents=[retail_data_agent, trending_fashion_agent],
)

# --- 2. Create Sequential Pipeline to gather info in parallel, then synthesize ---
root_agent = SequentialAgent(
    name="fashion_agent",
    sub_agents=[system_info_gatherer, fashion_report_synthesizer],
)

In [22]:
USER_ID = "CUST-001"

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=root_agent, app_name='fashion_advisor', session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [23]:
await run_session(
    runner,
    ["How can I increase my sales"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > How can I increase my sales


gemini-2.0-flash >  To provide you with the most effective strategies, I need a bit more information. Based on the retail data I have, here's a general approach, but please tell me more about what you're looking for so I can tailor my advice:

General Strategies based on the data:

*   **Focus on Best Sellers:** Increase marketing and promotional efforts around your "Highly Profitable" products (e.g., "Men's Denim Jacket," "Women's Floral Dress," "Unisex White Sneakers," "Women's Handbag"). Ensure you have sufficient stock to meet demand. Consider running targeted ads or offering bundles to boost sales further.
*   **Improve Underperforming Products:** Analyze why "Less Profitable" items like "Kids' Hoodie," "Men's Leather Belt," and "Women's Winter Coat" aren't performing as well. Is it the price point, the marketing, or the product itself? Consider discounts, updated product descriptions, or better placement in your store. For example, a seasonal item like the "Women's Winter Coat" m

In [24]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('fashion_advisor', 'test-db-session-01', 'user', '{"parts": [{"text": "How can I increase my sales"}], "role": "user"}')
('fashion_advisor', 'test-db-session-01', 'RetailDataAgent', '{"parts": [{"function_call": {"id": "adk-3dee1600-d404-4f2b-92c5-531a1a1e6fe4", "args": {}, "name": "get_retail_data"}}], "role": "model"}')
('fashion_advisor', 'test-db-session-01', 'RetailDataAgent', '{"parts": [{"function_response": {"id": "adk-3dee1600-d404-4f2b-92c5-531a1a1e6fe4", "name": "get_retail_data", "response": {"StoreSalesData": {"Store": {"Products": [{"ProductID": "P001", "Name": "Men\'s Denim Jacket", "Category": "Men", "Price": 79.99, "UnitsSold": 324, "Month": "2025-06", "ProfitCategory": "Highly Profitable"}, {"ProductID": "P002", "Name": "Women\'s Floral Dress", "Category": "Women", "Price": 49.99, "UnitsSold": 512, "Month": "2025-06", "ProfitCategory": "Highly Profitable"}, {"ProductID": "P003", "Name": "Unisex White Sneakers", "Categor

---
## Context Compaction

As you can see, all the events are stored in full in the session Database, and this quickly adds up. For a long, complex task, this list of events can become very large, leading to slower performance and higher costs.

But what if we could automatically summarize the past? Let's use ADK's **Context Compaction** feature to see **how to automatically reduce the context that's stored in the Session.**

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/context-compaction.png" width="1400" alt="Context compaction">

### Create an App for the agent

To enable this feature, let's use the same `root_agent` we created above. 

The first step is to create an object called `App`. We'll give it a name and pass in our root_agent. 

The Context Compaction feature uses a sliding window approach for collecting and summarizing agent workflow event data within a Session. When you configure this feature in your agent, it summarizes data from older events once it reaches a threshold of a specific number of workflow events, or invocations, with the current Session.

For this demo we will be using **`LlmEventSummarizer`** class allows you to specify a particular model for summarization.

We'll also create a new config to do the Context Compaction. This **`EventsCompactionConfig`** defines two key variables:

- **compaction_interval**: Asks the Runner to compact the history after every `n` conversations
- **overlap_size**: Defines the number of previous conversations to retain for overlap
- **summarizer**: LlmEventSummarizer object

We'll then provide this app to the Runner.

In [32]:
# Re-define our app with Events Compaction enabled
summarization_llm = Gemini(model="gemini-2.5-flash")
my_summarizer = LlmEventSummarizer(llm=summarization_llm)

fashion_app_compacting = App(
    name="fashion_app_compacting",
    root_agent=root_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        summarizer=my_summarizer,
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
fashion_runner_compacting = Runner(
    app=fashion_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

✅ Research App upgraded with Events Compaction!


/tmp/ipykernel_47/3899489539.py:9: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### Running the Demo

Now, let's have a conversation that is long enough to trigger the compaction. When you run the cell below, the output will look like a normal conversation. However, because we configured our `App`, a compaction process will run silently in the background after the 3rd invocation.

In the next step, we'll prove that it happened.

In [33]:
# Turn 1
await run_session(
    fashion_runner_compacting,
    "How can I increase my sales",
    "compaction_demo",
)

# Turn 2
await run_session(
    fashion_runner_compacting,
    "Give me products that can be displayed in winter",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    fashion_runner_compacting,
    "Does products from retail data can be displayed",
    "compaction_demo",
)

# Turn 4
await run_session(
    fashion_runner_compacting,
    "Does those products from retail data can give some profit",
    "compaction_demo",
)


 ### Session: compaction_demo

User > How can I increase my sales
gemini-2.0-flash >  Okay, let's synthesize all the information to create a clear action plan for increasing your sales, focusing on the winter display strategy and leveraging your existing retail data:

**I. Optimize Your Winter Display (Focus: Existing Inventory)**

*   **A. Women's Winter Coat (Transform from "Less Profitable"):**
    *   **Visual Appeal:**
        *   Ensure trending styles (puffer, longline, belted).
        *   Highlight water resistance, insulation, and stylish details.
        *   Use an attractive mannequin and accessories.
    *   **Incentives:**
        *   Offer a discount.
        *   Create bundles with scarves, gloves, and hats.
    *   **Inventory:** Prioritize new, trending styles.
*   **B. Men's Denim Jacket (Capitalize on "Highly Profitable"):**
    *   **Layering:** Display with sweaters and hoodies.
    *   **Winter Styling:** Add scarves and gloves.
    *   **Versatility:** Show how

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-195' coro=<_run_compaction_for_sliding_window() done, defined at /usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py:27> exception=ValueError("The last_update_time provided in the session object '2025-11-28 13:37:13' is earlier than the update_time in the storage_session '2025-11-28 13:37:26'. Please check if it is a stale session.")>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py", line 195, in _run_compaction_for_sliding_window
    await session_service.append_event(session=session, event=compaction_event)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/sessions/database_session_service.py", line 651, in append_event
    raise ValueError(
ValueError: The last_update_time provided in the session object '2025-11-28 13:37:13' is earlier than the update_time in the storage_session '2025-11-28 13:37:26'. Please ch

gemini-2.0-flash >  ## Consolidated Winter Display Products

Here's a consolidated list of products perfect for your winter display, combining trending insights and your retail data:

**I. Core Products (Leveraging Existing Stock)**

1.  **Women's Winter Coat:**
    *   **Why:** Essential winter item with potential for increased profitability.
    *   **Display Focus:** Trending styles (puffer, longline), warmth, water resistance, visual appeal with accessories. Make it the centerpiece!
2.  **Men's Denim Jacket:**
    *   **Why:** Extends the life of a popular item into winter through layering.
    *   **Display Focus:** Layered over sweaters/hoodies, accessorized with scarves/gloves.
3.  **Unisex White Sneakers:**
    *   **Why:** Year-round footwear that can be styled for winter.
    *   **Display Focus:** Paired with winter outfits, thicker socks, highlighting water resistance (if applicable).
4.  **Men's Casual T-Shirt (Long Sleeve):**
        *   **Why:** Promotes the sales of T-S

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-220' coro=<_run_compaction_for_sliding_window() done, defined at /usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py:27> exception=ValueError("The last_update_time provided in the session object '2025-11-28 13:37:26' is earlier than the update_time in the storage_session '2025-11-28 13:37:35'. Please check if it is a stale session.")>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py", line 195, in _run_compaction_for_sliding_window
    await session_service.append_event(session=session, event=compaction_event)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/sessions/database_session_service.py", line 651, in append_event
    raise ValueError(
ValueError: The last_update_time provided in the session object '2025-11-28 13:37:26' is earlier than the update_time in the storage_session '2025-11-28 13:37:35'. Please ch

gemini-2.0-flash >  **Absolutely, YES!**

The entire strategy is built around leveraging those products from your retail data to generate profit during the winter display.

Here's a recap of why and how:

*   **Women's Winter Coat:** Transforming this "less profitable" item into a profit center is a primary goal. This is achieved by:
    *   Ensuring it aligns with current trends.
    *   Maximizing visual appeal.
    *   Offering incentives (discounts, bundles).
*   **Men's Denim Jacket:** Capitalizing on its existing "highly profitable" status by:
    *   Extending its seasonal appeal through layering.
    *   Showcasing its versatility.
*   **Unisex White Sneakers:** Maintaining sales and profit by:
    *   Styling them appropriately for winter.
    *   Highlighting any winter-relevant features.
*   **Men's Casual T-Shirt (Long Sleeve):** Promotes the sales of T-Shirts during winter which in turn helps the sales of Denim Jacket

The success of the winter display depends on strategic

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-242' coro=<_run_compaction_for_sliding_window() done, defined at /usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py:27> exception=ValueError("The last_update_time provided in the session object '2025-11-28 13:37:35' is earlier than the update_time in the storage_session '2025-11-28 13:37:43'. Please check if it is a stale session.")>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/adk/apps/compaction.py", line 195, in _run_compaction_for_sliding_window
    await session_service.append_event(session=session, event=compaction_event)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/sessions/database_session_service.py", line 651, in append_event
    raise ValueError(
ValueError: The last_update_time provided in the session object '2025-11-28 13:37:35' is earlier than the update_time in the storage_session '2025-11-28 13:37:43'. Please ch

### Run completed

Above you can see that the app successfully executed all the questions

### Verifying Compaction in the Session History

The conversation above looks normal, but the history has been changed behind the scenes. How can we prove it?

We can inspect the `events` list from our session. The compaction process **doesn't delete old events; it replaces them with a single, new `Event` that contains the summary.** Let's find it.

In [35]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=fashion_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    # print(event)
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='02a6ed56-45ca-43ac-a752-c181843a1cf9' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1764336607.727001, 'end_timestamp': 1764337063.134686, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data'

### 4.4 What you've accomplished: Automatic Context Management

You just found the proof! The presence of that special summary `Event` in your session's history is the tangible result of the compaction process.

**Let's recap what you just witnessed:**

1.  **Silent Operation**: You ran a standard conversation, and from the outside, nothing seemed different.
2.  **Background Compaction**: Because you configured the `App` with `EventsCompactionConfig`, the ADK `Runner` automatically monitored the conversation length. Once the threshold was met, it triggered the summarization process in the background.
3.  **Verified Result**: By inspecting the session's events, you found the summary that the LLM generated. This summary now replaces the older, more verbose turns in the agent's active context.

**For all future turns in this conversation, the agent will be given this concise summary instead of the full history.** This saves costs, improves performance, and helps the agent stay focused on what's most important.


In [ ]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

---

| Authors |
| --- |
| [Jay Pathak](https://www.linkedin.com/in/jaypathak008/) |